# BayesOpt with a synthetic function

This is an example of Bayesian Optimization using `SimpleExperiment`, a subtype of `Experiment` used for experiments where parameter configurations can be evaluated synchrounously, without waiting for additional data.

In [1]:
import numpy as np
from ax import (
    ParameterType,
    RangeParameter,
    SearchSpace,
    SimpleExperiment, 
    modelbridge
)
from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.utils.notebook.plotting import render, init_notebook_plotting

In [2]:
init_notebook_plotting()

[INFO 03-15 19:17:58] ipy_plotting: Injecting Plotly library into cell. Do not overwrite or delete cell.


In [3]:
def branin(x1, x2):
    y = (x2 - 5.1 / (4 * np.pi ** 2) * x1 ** 2 + 5 * x1 / np.pi - 6) ** 2
    y += 10 * (1 - 1 / (8 * np.pi)) * np.cos(x1) + 10
    return y

First, we define an evaluation function that is able to compute all the metrics needed for this experiment. This function needs to accept a set of parameter values and a weight. It should produce a dictionary of metric names to tuples of mean and standard error for those metrics.

In [4]:
def branin_evaluation_function(
    parameterization, # dict of parameter names to values of those parameters
    weight=None, # evaluation function signature requires a weight argument
):
    if any(
        param_name not in parameterization.keys() for param_name in ["x1", "x2"]
    ):
        raise ValueError("Parametrization does not contain x1 or x2")
    x1, x2 = parameterization["x1"], parameterization["x2"]
    return {"branin": (branin(x1, x2), 0.0)}

Second, we define a search space for our experiment:

In [5]:
branin_search_space = SearchSpace(
    parameters=[
        RangeParameter(
            name="x1", parameter_type=ParameterType.FLOAT, lower=-5, upper=10
        ),
        RangeParameter(
            name="x2", parameter_type=ParameterType.FLOAT, lower=0, upper=15
        ),
    ]
)

Third, we make a `SimpleExperiment` — note that the `objective_name` needs to be one of the metric names returned by the evaluation function.

In [6]:
exp = SimpleExperiment(
    name="test_branin",
    search_space=branin_search_space,
    evaluation_function=branin_evaluation_function,
    objective_name="branin",
    minimize=True,
)

By default, the objective will be maximized in optimization; to change this behavior, pass `minimize=True` to `SimpleExperiment`. It is also possible to set outcome constraints for the `SimpleExperiment` by passing a list of `OutcomeConstraint` as the `outcome_constraints` keyword argument.

In [7]:
print(f"Running Sobol initialization batches...")
# Sobol generator does not require experiment data, so we only instantiate it once.
sobol = modelbridge.get_sobol(exp.search_space)
for i in range(5):
    exp.new_trial(generator_run=sobol.gen(1))
    
for i in range(15):
    print(f"Running GP+EI optimization batch {i+1}/15...")
    # GP+EI model requires experiment data to re-fit the GP, so we reinstantiate 
    # it with every trial, as every trial adds new data to the experiment.
    gpei = modelbridge.get_GPEI(experiment=exp, data=exp.eval())
    batch = exp.new_trial(generator_run=gpei.gen(1))
    
print("Done!")

Running Sobol initialization batches...
Running GP+EI optimization batch 1/15...
Running GP+EI optimization batch 2/15...
Running GP+EI optimization batch 3/15...
Running GP+EI optimization batch 4/15...
Running GP+EI optimization batch 5/15...


/data/users/kkashin/fbsource/fbcode/buck-out/dev/gen/bento/kernels/bento_kernel_ae#link-tree/gpytorch/utils/cholesky.py:41: RuntimeWarning:

A not p.d., added jitter of 1e-08 to the diagonal



Running GP+EI optimization batch 6/15...
Running GP+EI optimization batch 7/15...
Running GP+EI optimization batch 8/15...
Running GP+EI optimization batch 9/15...
Running GP+EI optimization batch 10/15...
Running GP+EI optimization batch 11/15...
Running GP+EI optimization batch 12/15...
Running GP+EI optimization batch 13/15...
Running GP+EI optimization batch 14/15...
Running GP+EI optimization batch 15/15...
Done!


Now we can inspect the `SimpleExperiment`'s data by calling `eval()`, which retrieves evaluation data for all batches of the experiment. Note that if using `Experiment` instead of `SimpleExperiment`, you will need to use `fetch_data()` instead of `eval()`.


In [8]:
exp.eval().df

,arm_name,mean,metric_name,sem,trial_index
0,0_0,21.881901,branin,0.0,0
1,1_0,58.260392,branin,0.0,1
2,2_0,18.291666,branin,0.0,2
3,3_0,108.445979,branin,0.0,3
4,4_0,32.456973,branin,0.0,4
5,5_0,3.333691,branin,0.0,5
6,6_0,9.752795,branin,0.0,6
7,7_0,10.960889,branin,0.0,7
8,8_0,23.376577,branin,0.0,8
9,9_0,250.741219,branin,0.0,9


We can also use the `eval_trial` function to get evaluation data for a specific trial in the experiment, like so:

In [9]:
trial_data = exp.eval_trial(exp.trials[1])
trial_data.df

,arm_name,mean,metric_name,sem,trial_index
0,1_0,58.260392,branin,0.0,1


Now we can plot the results of our optimization:

In [10]:
render(plot_contour(model=gpei, param_x='x1', param_y='x2', metric_name='branin', relative=False))

In [11]:
# `plot_single_method` expects a 2-d array of means, because it expects to average means from multiple 
# optimization runs, so we wrap out best objectives array in another array.
best_objectives = np.array([[trial.objective_mean for trial in exp.trials.values()]])
best_objective_plot = optimization_trace_single_method(
        y=np.minimum.accumulate(best_objectives, axis=1),
        optimum=0.397887,  # Known minimum objective for Branin function.
)
render(best_objective_plot)